In [3]:
__author__ = "sai koneru (sdk96@psu.edu)"
# TO DO: Find a way to save the results if there is any error in between. 
# TO DO: Give the info on data quality at end and if too bad may be a mail notificatio?


import logging
import os 
from mastodon import Mastodon
from migration import utils 
import pandas as pd
from datetime import datetime
from pathlib import Path
from tqdm import tqdm
from migration.utils import async_requests
import requests
import json
from datetime import datetime
import re
import time

logger = utils.setup_logger(module = __name__ ,
                         file = f'../logs/{__name__}.log')

logger.info(' creating the required authentication variables')

Mastodon.create_app(
     'migrationapp',
     api_base_url = 'https://mastodon.social',
     to_file = '../secrets/migrationapp.secret'
)

mastodon = Mastodon(
    client_id = '../secrets/migrationapp.secret',
    api_base_url = 'https://mastodon.social'
)

mastodon_usr = os.environ.get('MASTODON_USER')
mastodon_pwd = os.environ.get('MASTODON_PWD')

access_token = mastodon.log_in(
    username = mastodon_usr,
    password = mastodon_pwd,
    to_file = '../secrets/pytooter_usercred.secret'
)


logger.info('collecting mastodon user names!')

2024-09-17 18:05:23,684:3146192478:3146192478.py:<module>:INFO creating the required authentication variables
2024-09-17 18:05:23,684:3146192478:3146192478.py:<module>:INFO creating the required authentication variables
2024-09-17 18:05:25,159:3146192478:3146192478.py:<module>:INFOcollecting mastodon user names!
2024-09-17 18:05:25,159:3146192478:3146192478.py:<module>:INFOcollecting mastodon user names!


In [4]:
# read all the csvs and combine them into one
accounts_dir = Path('../data/raw/mastodonAccounts/')
csvs = accounts_dir.glob('*csv')
dfs = []
for csv in tqdm(csvs):    
    dfs.append(pd.read_csv(csv)[['Account address']])

accounts_df = pd.concat(dfs).reset_index(drop=True)

#remove any nan values for now
accounts_df.dropna(inplace=True)

50it [00:00, 211.19it/s]


In [5]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token
    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

auth = BearerAuth(access_token)

In [6]:
import requests
requests.get('https://mastodon.example/api/v1/accounts/14715/followers?limit=20000&max_id=7486869', auth=auth)

ConnectionError: HTTPSConnectionPool(host='mastodon.example', port=443): Max retries exceeded with url: /api/v1/accounts/14715/followers?limit=20000&max_id=7486869 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000022A74F5BE20>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [37]:
index

6290

In [45]:
from tqdm import tqdm
accounts_df['account_id'] = ''
for index, row in tqdm(accounts_df[9253:].iterrows()):
    account_id = mastodon.account_search(row['Account address'])
    if account_id:
        accounts_df.at[index, 'account_id'] = account_id[0]['id']

1599it [24:29,  1.09it/s]


In [50]:
(accounts_df.to_csv('account_ids_mapped.csv'))

In [51]:
(accounts_df.to_pickle('account_ids_mapped.pkl'))

In [28]:
row['Account address']

'@LauraSikstrom@mastodon.social'

In [14]:
mastodon.account_followers(id=147151)

[{'id': 39690,
  'username': 'Niles',
  'acct': 'Niles',
  'display_name': 'King Niles',
  'locked': False,
  'bot': False,
  'discoverable': None,
  'indexable': False,
  'group': False,
  'created_at': datetime.datetime(2017, 4, 3, 0, 0, tzinfo=tzutc()),
  'note': '<p>Networking Enthusiast/Aspiring System Administrator/Networking and Cloud Tech Student <a href="https://mastodon.social/tags/Networking" class="mention hashtag" rel="tag">#<span>Networking</span></a> <a href="https://mastodon.social/tags/InfoSec" class="mention hashtag" rel="tag">#<span>InfoSec</span></a></p>',
  'url': 'https://mastodon.social/@Niles',
  'uri': 'https://mastodon.social/users/Niles',
  'avatar': 'https://files.mastodon.social/accounts/avatars/000/039/690/original/c41a9fbb3e4fb125.jpg',
  'avatar_static': 'https://files.mastodon.social/accounts/avatars/000/039/690/original/c41a9fbb3e4fb125.jpg',
  'header': 'https://mastodon.social/headers/original/missing.png',
  'header_static': 'https://mastodon.social

In [7]:
from tqdm import tqdm
accounts_df['account_id'] = ''
for index, row in tqdm(accounts_df.iterrows()):
    account_id = mastodon.account_search(row['Account address'])
    if account_id:
        accounts_df.at[index, 'account_id'] = account_id[0]['id']

,Account address
0,@anaivas@mastodon.ie
1,@Anarchopology@kolektiva.social
2,@anibalarregui@mastodon.world
3,@anthrobirder@mstdn.social
4,@anthropam@climatejustice.social
...,...
11299,cass@user
11300,@VanOorschot_FEST@freiburg.social
11301,@as_sAlone@mastodon.social
11302,@dapo63@social.anoxinon.de


In [33]:
errors = []

def extract_account_info(data):
    try:
        responses = data.json()['accounts']
        if responses: 
            return {'user_id': responses[0]['id']}
        else:
            return {'user_id': '000000000000'}
        
    except:
        errors.append(data)
        return {'user_id': '000000000000'}

# checking if there are any inconsistent entries to the data
inconsistents = accounts_df[~accounts_df['Account address'].str.startswith('@')]
consistents = accounts_df[accounts_df['Account address'].str.startswith('@')]

logger.info(f' ugh there seems to be some data inconsistencies for atleast {len(inconsistents)} accounts and {len(consistents)} consistent account names') 
logger.info(' working on recovering the inconsistent accounts')

inc_accounts = []
for index, row in tqdm(inconsistents.iterrows()):
    try:
        if row['Account address'].startswith('http'):
            resp = requests.get(row['Account address'], auth=auth)
            res = re.search('\([^)]*\)', resp.text)
            inc_accounts.append(res.group()[1:-1])
        else:
            inc_accounts.append('@'+row['Account address'])
    except:
        continue

inconsistents = pd.DataFrame(inc_accounts, columns=['Account address'])

logger.info(f' able to partially recover {len(inconsistents)} this doesnt gaurentee a result though')

accounts_df = pd.concat([consistents, inconsistents])
urls = "https://mastodon.social/api/v2/search?q="+accounts_df['Account address'].str.strip()+"&resolve=true&limit=1"

logger.info(' sleeping for a few seconds to be careful about too many request error')
time.sleep(5)

logger.info(' making the api queries!')

result = async_requests.async_request_url_list(auth=auth,
                                      data_processing_fn=extract_account_info,
                                      use_tqdm=True,
                                      max_workers=6,
                                      urls=urls,
                                      rate_limit_interval_secs=6,
                                     )
                                         
    #return result

#if __name__ == '__main__':
    

2024-09-17 18:46:28,659:2236657696:2236657696.py:<module>:INFO ugh there seems to be some data inconsistencies for atleast 0 accounts and 10852 consistent account names
2024-09-17 18:46:28,659:2236657696:2236657696.py:<module>:INFO ugh there seems to be some data inconsistencies for atleast 0 accounts and 10852 consistent account names
2024-09-17 18:46:28,660:2236657696:2236657696.py:<module>:INFO working on recovering the inconsistent accounts
2024-09-17 18:46:28,660:2236657696:2236657696.py:<module>:INFO working on recovering the inconsistent accounts
0it [00:00, ?it/s]
2024-09-17 18:46:28,667:2236657696:2236657696.py:<module>:INFO able to partially recover 0 this doesnt gaurentee a result though
2024-09-17 18:46:28,667:2236657696:2236657696.py:<module>:INFO able to partially recover 0 this doesnt gaurentee a result though
2024-09-17 18:46:28,675:2236657696:2236657696.py:<module>:INFO sleeping for a few seconds to be careful about too many request error
2024-09-17 18:46:28,675:223665

In [34]:
result

[{'user_id': '109296046670778490', 'idx': 4},
 {'user_id': '109387309542862655', 'idx': 2},
 {'user_id': '109292347780006776', 'idx': 0},
 {'user_id': '109287404708135353', 'idx': 3},
 {'user_id': '000000000000', 'idx': 1},
 {'user_id': '109245996633387829', 'idx': 5},
 {'user_id': '109316268633951069', 'idx': 8},
 {'user_id': '109245896155043487', 'idx': 7},
 {'user_id': '108198799780351729', 'idx': 10},
 {'user_id': '109316681744743868', 'idx': 9},
 {'user_id': '109306399565151071', 'idx': 6},
 {'user_id': '109267482710594684', 'idx': 11},
 {'user_id': '109378945970544547', 'idx': 12},
 {'user_id': '108194828161896328', 'idx': 13},
 {'user_id': '109290571070706310', 'idx': 15},
 {'user_id': '000000000000', 'idx': 14},
 {'user_id': '109387309657068160', 'idx': 16},
 {'user_id': '109258207542393959', 'idx': 17},
 {'user_id': '000000000000', 'idx': 19},
 {'user_id': '109267071649334218', 'idx': 20},
 {'user_id': '000000000000', 'idx': 18},
 {'user_id': '109277126344150593', 'idx': 21},
